In [ ]:
# must run on activated terminal
# i.e source activate py310
!pip install --user transformers[torch] datasets evaluate
!pip install --user torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116

In [3]:
!pip install wandb

  Using cached wandb-0.15.4-py3-none-any.whl (2.1 MB)
  Using cached setproctitle-1.3.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30 kB)
  Using cached pathtools-0.1.2-py3-none-any.whl
  Using cached sentry_sdk-1.25.1-py2.py3-none-any.whl (206 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)


In [1]:
from transformers import AutoTokenizer
# using disilroberta as a quick baseline model
# https://huggingface.co/docs/transformers/tasks/sequence_classification
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)

In [ ]:
feature_df

,full_caption,username,Openness,Conscientiousness,Extroversion,Agreeableness,Neuroticism
59,caption did you know henry ford made the 8 hou...,36mktg,2,2,2,2,0
69,caption so clean smiling_face_with_heart-eyesr...,6staroileaters,0,2,1,1,1
26,caption through the night crescent_moon • maet...,740project,2,0,0,0,1
57,caption take your social media accounts out of...,_boostfruit.com_,2,0,2,1,1
10,caption appreciate you excellencealways folded...,_cloutfinder,2,0,2,1,1
...,...,...,...,...,...,...,...
71,"caption none, is of type image, has 1 comments...",wipmarketing,2,2,2,2,0
14,caption enjoy this video while i disappear for...,xcarmeeen,0,0,2,0,2
5461,"caption power back.#fanggang | xfl, is of type...",xflvipers,0,2,2,2,0
67,caption summeeeeer! all the hard work of clean...,yogabody_uk,2,2,2,2,0


In [ ]:
feature_df[feature_df['Openness']==0]['full_caption']

69      caption so clean smiling_face_with_heart-eyesr...
5490    caption best day of my life 🫶heart_suit️jetzt ...
22      caption memorial day weekenTongue_sticking_out...
5484    caption mission accomplished. mold broken. i l...
47      caption anyone got a pre-order on one of these...
                              ...                        
108     caption logo variations thinking_face.....#dig...
24      caption wah taj!!#taj 2nd season#congratulatio...
136     caption it’s in our dna.big love,tbx, is of ty...
14      caption enjoy this video while i disappear for...
5461    caption power back.#fanggang | xfl, is of type...
Name: full_caption, Length: 66, dtype: object

In [ ]:
feature_df.to_parquet("feature.parquet")

In [ ]:
pd.options.display.max_colwidth=100
feature_df[feature_df['Openness']==0].full_caption.iloc[1]

'caption best day of my life \U0001faf6heart_suit️jetzt überall erhältlich firehundred_pointsfolded_hands•clip by: filmschieber_agency > _luck3 track by: komacasper x lucawolf_ #komacasper #vocaltekkno #vocaltekknoworldwide #lucawolf #ssb #festival #rave, is of type video, has 37 comments and 1804.0 likescaption sputnik springbreak 2023 es war absolut grandios smiling_face_with_heart-eyeshundred_pointsdankeschön für dieses unvergessliche erlebnis, ihr habt es zu dem gemacht was es war 6000 menschen die bock auf vocaltekkno hatten! jeder der dabei war darf sich persönlich von mir gedrückt fühlen heart_suit️folded_handsdankeschön bis 2024 double_exclamation_mark️smiling_face_with_halohundred_points•video by: filmschieber_agency > _luck3 #komacasper #vocaltekkno #vocaltekknoworldwide #sputnik #festival #ssb #julianluckeproductions #rave, is of type video, has 65 comments and 3522.0 likescaption der sommer ist da und hier sind meine dates für juni und juli 2023 fire\U0001faf6wir sehen uns 

In [2]:
!pip install wandb

  Using cached wandb-0.15.4-py3-none-any.whl (2.1 MB)
  Using cached sentry_sdk-1.25.1-py2.py3-none-any.whl (206 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached pathtools-0.1.2-py3-none-any.whl
  Using cached setproctitle-1.3.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)


In [ ]:
feature_df['Openness'].value_counts()

2.0    200
0.0     66
Name: Openness, dtype: int64

In [ ]:
feature_df['Conscientiousness'].value_counts()

0.0    61
2.0     1
Name: Conscientiousness, dtype: int64

In [ ]:
feature_df['Extroversion'].value_counts()

2    229
0     27
1     10
Name: Extroversion, dtype: int64

In [ ]:
feature_df['Agreeableness'].value_counts()

2    139
0     82
1     45
Name: Agreeableness, dtype: int64

In [ ]:
feature_df['Neuroticism'].value_counts()

0    194
1     45
2     27
Name: Neuroticism, dtype: int64

In [2]:
from collections import defaultdict
# Define 2 preprocessing functions
def tokenize_input(examples):
    return tokenizer(examples["full_caption"])

def group_texts(examples):
    block_size = 128    
    
    result= defaultdict(list)
    for i, input_id  in enumerate(examples['input_ids']):
        attn_mask = examples['attention_mask'][i]
        total_length = len(input_id)
        if total_length > block_size:
            total_length = (total_length // block_size) * block_size
        num_chunks = total_length // block_size
        result['input_ids'].extend([input_id[i : i + block_size] for i in range(0, total_length, block_size)])
        result['attention_mask'].extend([attn_mask[i : i + block_size] for i in range(0, total_length, block_size)])
        result['Openness'].extend([examples['Openness'][i] for _ in range(num_chunks)])
        result['Conscientiousness'].extend([examples['Conscientiousness'][i] for _ in range(num_chunks)])
        result['Extroversion'].extend([examples['Extroversion'][i] for _ in range(num_chunks)])
        result['Agreeableness'].extend([examples['Agreeableness'][i] for _ in range(num_chunks)])
        result['Neuroticism'].extend([examples['Neuroticism'][i] for _ in range(num_chunks)])    

    return result


from datasets import Dataset
import pandas as pd 
all_five = ['Openness', 'Conscientiousness','Extroversion', 'Agreeableness', 'Neuroticism']
feature_df = pd.read_parquet("feature.parquet")
feature_df = feature_df.dropna()

d = {0.5:1,1.0:2,0:0}
for column in all_five:
    feature_df[column] = feature_df[column].map(d)
    feature_df.astype({column:'int32'}).dtypes
focused_trait = "Agreeableness"
all_five.remove(focused_trait)

ds = Dataset.from_dict(feature_df.to_dict('list'))
tokenized_ds = ds.map(tokenize_input, batched=True, num_proc=4, remove_columns=['full_caption','username'])
lm_dataset_ig = tokenized_ds.map(group_texts, num_proc=4, batched=True)
lm_dataset_ig = lm_dataset_ig.shuffle(seed=42)
lm_dataset_ig = lm_dataset_ig.rename_column(focused_trait, "label")
lm_dataset_ig = lm_dataset_ig.remove_columns(all_five)
lm_dataset_ig = lm_dataset_ig.train_test_split(test_size=0.2)
lm_dataset_ig

Map (num_proc=4):   0%|          | 0/266 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (13472 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3007 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (25039 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4864 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/266 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 8977
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 2245
    })
})

In [ ]:
lm_dataset_ig['train']['label']

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,


### Dataset IO

In [ ]:
lm_dataset_ig.save_to_disk("distilbert_base_block128_train_test_split_tokenized_feature")

Saving the dataset (0/1 shards):   0%|          | 0/8977 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2245 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk
lm_dataset_ig = load_from_disk("distilbert_base_block128_train_test_split_tokenized_feature")

In [ ]:
# rename Agreeableness to label

In [ ]:
lm_dataset_ig['train']['label']

[2,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 0,


In [ ]:

arr = [14, 31, 27, 27, 31, 13, 14, 13]
stats.itemfreq(arr)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  This is separate from the ipykernel package so we can avoid doing imports until


array([[13,  2],
       [14,  2],
       [27,  2],
       [31,  2]])

In [4]:
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from scipy import stats

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    print(f"predictions:{predictions}, item_freq:{stats.itemfreq(predictions)}.")
    print(f"labels:{labels}, item_freq:{stats.itemfreq(labels)}.")
    accuracy_measurement =  accuracy.compute(predictions=predictions, references=labels)
    precision_measurement =  precision.compute(predictions=predictions, references=labels, average='micro')
    recall_measurement =  recall.compute(predictions=predictions, references=labels, average='micro')
    print(f"Accuracy:{accuracy_measurement}, Precision:{precision_measurement}, Recall:{recall_measurement}")
    return precision_measurement

id2label = {0: "LOW", 1: "VAGUE", 2: "HIGH"}
label2id = {"LOW": 0, "VAGUE":1, "HIGH": 2}

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)
import os 
os.environ["WANDB_PROJECT"]="upwork-ocean-classification"
os.environ["WANDB_LOG_MODEL"]="true"
os.environ["WANDB_WATCH"]="false"

training_args = TrainingArguments(
    output_dir=f"distilbert-base-uncased-{focused_trait}-classification-shuffled",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="no",
    report_to="wandb",
    push_to_hub=False,
    run_name=focused_trait
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset_ig["train"],
    eval_dataset=lm_dataset_ig["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,

)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.

In [2]:
try:
    import wandb
    from wandb import init, log, join  # test that these are available
except ImportError:
   print("msg")

msg


In [4]:
import wandb
wandb.__path__

ModuleNotFoundError: No module named 'wandb'

In [5]:
import wandb

ModuleNotFoundError: No module named 'wandb'

In [5]:
import wandb
trainer.train(True)
wandb.finish()

NameError: name 'trainer' is not defined

In [27]:
trainer.state

TrainerState(epoch=4.0, global_step=2248, max_steps=562, num_train_epochs=1, total_flos=1189181044712448.0, log_history=[{'epoch': 0.89, 'learning_rate': 1.5551601423487547e-05, 'loss': 0.759, 'step': 500}, {'epoch': 1.0, 'eval_loss': 0.5201415419578552, 'eval_precision': 0.7741648106904232, 'eval_runtime': 9.1667, 'eval_samples_per_second': 244.909, 'eval_steps_per_second': 15.382, 'step': 562}, {'epoch': 1.78, 'learning_rate': 1.110320284697509e-05, 'loss': 0.4035, 'step': 1000}, {'epoch': 2.0, 'eval_loss': 0.39854949712753296, 'eval_precision': 0.8503340757238307, 'eval_runtime': 9.0941, 'eval_samples_per_second': 246.865, 'eval_steps_per_second': 15.505, 'step': 1124}, {'epoch': 2.67, 'learning_rate': 6.654804270462634e-06, 'loss': 0.2045, 'step': 1500}, {'epoch': 3.0, 'eval_loss': 0.4845384359359741, 'eval_precision': 0.8538975501113586, 'eval_runtime': 9.0892, 'eval_samples_per_second': 246.995, 'eval_steps_per_second': 15.513, 'step': 1686}, {'epoch': 3.56, 'learning_rate': 2.20

## inference!

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
ft_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-Openess-classification-shuffled-dataset/checkpoint-1124")

In [ ]:
from transformers import AutoTokenizer, DistilBertModel, DistilBertForSequenceClassification
import torch

# when sampling some 
inputs = tokenizer(feature_df[feature_df[focused_trait]==0].full_caption.iloc[12][100:200], return_tensors="pt")


with torch.no_grad():
    logits = ft_model(**inputs).logits
    
logits

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6                                                                                             │
│    7                                                                                             │
│    8 with torch.no_grad():                                                                       │
│ ❱  9 │   logits = ft_model(**inputs).logits                                                      │
│   10                                                                                             │
│   11 logits                                                                                      │
│   12                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'ft_model' is not defined

In [ ]:
predicted_class_id = logits.argmax().item()
predicted_class_id

2

In [ ]:
outputs.logits

In [ ]:
logits

tensor([[ 0.1415,  0.1903, -0.0875]])

In [ ]:
labels = torch.tensor([1])
loss = model(**inputs, labels=labels).loss

In [ ]:
predicted_class_id = logits.argmax().item()

In [ ]:
predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


In [ ]:
!pip install --user transformers

  Using cached transformers-4.30.1-py3-none-any.whl (7.2 MB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!python3 --version

Python 3.7.12


## Example from 
https://huggingface.co/docs/transformers/tasks/language_modeling#preprocess

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    print(concatenated_examples.keys())
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


In [ ]:
from datasets import load_dataset

eli5 = load_dataset("eli5", split="train_asks[:5000]")
eli5 = eli5.flatten()
eli5 = eli5.train_test_split(test_size=0.2)
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

Found cached dataset eli5 (/home/jupyter/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa)


Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (693 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1311 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (651 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (982 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1483 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (618 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (801 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1161 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
tokenized_eli5.column_names

{'train': ['input_ids', 'attention_mask'],
 'test': ['input_ids', 'attention_mask']}

In [ ]:
#tokenized_eli5.keys()
sum(tokenized_ds['train'], [])

TypeError: can only concatenate list (not "dict") to list

In [ ]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])


In [ ]:
len(lm_dataset['train'][0]['input_ids'])

128

In [ ]:
tokenized_eli5['train']

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 4000
})

In [ ]:
from transformers import DataCollatorForLanguageModeling
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="my_awesome_eli5_clm-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

2023-06-07 19:41:22.795413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 19:41:25.997772: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-06-07 19:41:25.997902: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

RuntimeError: Failed to import transformers.data.data_collator because of the following error (look up to see its traceback):
/opt/conda/lib/python3.10/site-packages/torch/lib/libtorch_cuda.so: undefined symbol: cudaGraphInstantiateWithFlags, version libcudart.so.11.0